In [21]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import requests

IndentationError: expected an indented block (<ipython-input-21-74065ae729ce>, line 2)

In [6]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\dell\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


# NASA Mars News

In [7]:
url = 'https://mars.nasa.gov/news'

# Retrieve page with the requests module
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
news_soup = BeautifulSoup(html, 'html.parser')


# Retrieve the parent divs for all articles
articles = news_soup.find("li", class_ = "slide")
news_title = articles.find('div', class_='content_title').text
news_p = articles.find('div', class_='article_teaser_body').text

# JPL Mars Space Images - Featured Image

In [9]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Retrieve page with the requests module
browser.visit(url)
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
image_soup = BeautifulSoup(html, 'html.parser')
browser.links.find_by_partial_text('FULL IMAGE').click()

# # Create BeautifulSoup object; parse with 'html.parser' for the full image page
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

#frame = image_soup.find("div", class_ = 'fancybox-lock')
image = image_soup.find("img", class_ = "fancybox-image")['src']

base_url = 'https://www.jpl.nasa.gov'
image_link = f'{base_url}{image}'
image_link


'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17563_ip.jpg'

# Mars Facts

In [10]:
url = 'https://space-facts.com/mars/'

In [11]:
tables = pd.read_html(url)


In [12]:
description = tables[0]
description.columns = ["Description", "Mars"]
description

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
description_html = description.to_html(index=False, border=0, classes = 'table table-striped')
description_html

'<table border="0" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [15]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# Retrieve page with the requests module
browser.visit(url)
html = browser.html

#Use Beautiful soup to find Hemisphere titles in text
elements_soup = BeautifulSoup(html, 'html.parser')
elements_div = elements_soup.find("div", class_ = "collapsible")
elements = elements_div.find_all("h3")
elements = [element.text for element in elements]
elements

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [16]:
# Use Splinter to click each title and scrap for the full hemisphere image

hemisphere_image_urls = [] # empty list to be appended by dictionaries

for x in range(len(elements)): #parse for each title element
    
    browser.links.find_by_partial_text(f'{elements[x]}').click()
    
    title = elements[x]
    image_url = browser.find_by_css('a').links.find_by_partial_text("Sample")['href']
    
    hemisphere_image_urls.append(
                                    {"title": f'{title}', "img_url": f'{image_url}'}
                                )
#return to home page
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    html = browser.html
    

In [17]:
browser.quit()

In [19]:
dict_final = {"news_title":news_title,
             "news_p":news_p,
              "featured_image": image_link,
              "table": description_html,
              "hemispheres":hemisphere_image_urls      
             }


{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_p': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17563_ip.jpg',
 'table': '<table border="0" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Or